# Links

- https://github.com/bitcoinbook/bitcoinbook
- https://github.com/jimmysong/programmingbitcoin


ledger docs on HD wallets - https://developers.ledger.com/docs/nano-app/psd-applications/


![where-are-my-assets](https://developers.ledger.com/docs/nano-app/images/where_are_my_assets.png "Where are my assets?")


# Libs

- https://github.com/trezor/python-mnemonic


# BIPS

### HD wallets

- https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki
- https://github.com/bitcoin/bips/blob/master/bip-0039.mediawiki
- https://github.com/bitcoin/bips/blob/master/bip-0043.mediawiki
- https://github.com/bitcoin/bips/blob/master/bip-0044.mediawiki


In [2]:
"""
Generate a Bitcoin address
"""

import secrets

# generate random 256 bit (32 bytes) hex token
secret = secrets.token_hex(32)


def generate_address(network="mainnet", invoice_type="P2PKH"):
    """
    network: 'mainnet' or 'testnet'
    invoice_type: 'P2PKH', 'P2SH', or 'bech32'
    """
    # generate random 256 bit unsigned int
    privkey = secrets.randbits(256)
    
    if invoice_type.lower() == "p2pkh":
        pass
    else:
        raise NotImplementedError()
    

generate_address()

In [3]:
base58_alphabet = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"
# https://en.bitcoinwiki.org/wiki/Base58

base64_alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
# https://en.wikipedia.org/wiki/Base64

In [6]:
import ecdsa

signing_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
secret = signing_key.privkey.secret_multiplier
print(f"privkey: {secret.to_bytes(32, 'big').hex()}")

verifying_key = signing_key.verifying_key

(x, y) = verifying_key.pubkey.point.x(), verifying_key.pubkey.point.y()
print(f"x: {x.to_bytes(32, 'big').hex()}")
print(f"y: {y.to_bytes(32, 'big').hex()}")

privkey: 1388bf135d04fbcb5efe4da2ba48da3db738623a0a804713eb99a5ace40df95c
x: 007f3bfec9ba050cbda63077e31dd33b04cc978ddb6f9542f30f4b0f8cf1f759
y: 32526018b68c26cf259010fa26a45fc1238c6ae94717919e40ef6073447490fc


In [4]:
import hashlib
from base58 import b58encode, b58encode_check

def bitcoin_hash(x: int, y: int, compressed=False):
    x_bytes = x.to_bytes(32, 'big')
    y_bytes = y.to_bytes(32, 'big')

    if compressed:
        # https://karpathy.github.io/2021/06/21/blockchain/
        prefix = b"\x02" if y % 2 == 0 else b"\x03"
        ux_pubkey = prefix + x_bytes
    else:
        ux_pubkey = b"\x04" + x_bytes + y_bytes

    hash256 = hashlib.sha256(ux_pubkey).digest()
    ripe_hash = hashlib.new("ripemd160", hash256).digest()
    return ripe_hash

def to_bitcoin_address(
    x: int,
    y: int,
    network: str="mainnet",
    compressed: bool=False
) -> bytes:
    # https://en.bitcoin.it/w/images/en/9/9b/PubKeyToAddr.png

    ripe_hash = bitcoin_hash(x, y, compressed=compressed)

    if network == "mainnet":
        network_id = b"\x00"
    elif network == "testnet":
        network_id = b"\x6f"
    else:
        raise ValueError("network must be 'mainnet' or 'testnet'")
    
    ux_hash = network_id + ripe_hash
    double_sha = hashlib.sha256(hashlib.sha256(ux_hash).digest()).digest()

    ux_addr = ux_hash + double_sha[:4]
    return b58encode(ux_addr)  # why not b58encode_check ?

#addr = to_bitcoin_address(x, y, network="mainnet")
#print(addr, len(addr))

In [26]:
def generate_bitcoin_keys_address(network: str="testnet", compressed: bool=True):
    
    signing_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    secret = signing_key.privkey.secret_multiplier

    verifying_key = signing_key.verifying_key

    (x, y) = verifying_key.pubkey.point.x(), verifying_key.pubkey.point.y()

    addr = to_bitcoin_address(x, y, network=network, compressed=compressed)
    return secret, (x, y), addr

keys_1 = generate_bitcoin_keys_address()
print(keys_1)
keys_2 = generate_bitcoin_keys_address()
print(keys_2)

(77650971224124458173331944410079081514457453147226445995403992195744994942909, (93684660838336764568736530954706445561087138452135284275701897802049104472422, 46649374848673110858765950369998988444931853720143716968769917941804667161586), b'mmJRzDNfx3aYFiQmrCbbrKD4nzfqxtsiiN')
(63496753375946996063030423098218736103448894552165900619530569956968829940868, (88612169415003993578166821014637966596834433723705348461664491846313666282016, 105335839458094758109125901264188257602710252056533238647914758144945593138896), b'mi54iWdghdHMaipYfZFzJxmvMbtyb3fQkp')


In [6]:
import hashlib
keys_1 = (77650971224124458173331944410079081514457453147226445995403992195744994942909, (93684660838336764568736530954706445561087138452135284275701897802049104472422, 46649374848673110858765950369998988444931853720143716968769917941804667161586), b'mmJRzDNfx3aYFiQmrCbbrKD4nzfqxtsiiN')

ripe_hash = bitcoin_hash(keys_1[1][0], keys_1[1][1], compressed=True)
print(ripe_hash.hex())

3f7276b2b30d44c67685406910b9669b79e81d30


In [7]:
# adapted from
# https://karpathy.github.io/2021/06/21/blockchain/
from dataclasses import dataclass

@dataclass
class TxIn:
    prev_tx: bytes
    prev_index: int
    script_sig: Script = None
    sequence: int = 0xffffffff

tx_in_1 = TxIn(
    # https://blockstream.info/testnet/address/mmJRzDNfx3aYFiQmrCbbrKD4nzfqxtsiiN
    prev_tx=bytes.fromhex("2d67a5b64bbfb2d8f26780f2580de0a297234d9fa3dafaae93a9192828adada8"),
    prev_index=1
)

@dataclass
class TxOut:
    amount: int  # in satoshis
    script_pubkey: Script = None

# tx_in_1 is a utxo of 10,000 satoshis
tx_out1 = TxOut(amount=1000)
tx_out2 = TxOut(amount=9000)


NameError: name 'Script' is not defined

In [8]:
# https://github.com/bitcoin/bitcoin/blob/master/src/script/script.h
SCRIPT_OPCODES_MAP = {
    "OP_DUP": 0x76,
    "OP_HASH160": 0xA9,
    # "OP_PUSHBYTES_20": 0x00,  # ???
    "OP_EQUALVERIFY": 0x88,
    "OP_CHECKSIG": 0xAC,
}

In [10]:
script_pubkey = "001427801ec818bff0facf4754cc9ca71900bce2282d"
len(script_pubkey)

44